In [ ]:
import pandas as pd
pd.set_option("display.max_rows", 500)

In [ ]:
raw = pd.read_csv("https://data.cdc.gov/api/views/2ew6-ywp6/rows.csv?accessType=DOWNLOAD")
counties = pd.read_csv("https://raw.githubusercontent.com/covid-projections/covid-data-model/main/data/misc/fips_population.csv")[["fips", "population"]]

In [ ]:
# counties["fips"] = counties["fips"].astype(str).str.zfill(5)
raw.county_fips.unique()

In [ ]:
fpl_counties = ["12099", "12086", "12011", "12033", "12111", "19113", "55071", "33015", "48201", "06075", "27123", "27053"]
split = raw["county_names"].str.split(",")

In [ ]:
# fpl_data = raw.loc[raw["county_fips"].isin(fpl_counties)].copy()
fpl_data = raw
fpl_data["fips"] = fpl_data["county_fips"].astype(str).str.zfill(5)

In [ ]:
# idx = fpl_data.groupby(['county_fips'])['date_start'].transform(max) == fpl_data['date_start']
# # fpl_data[idx]
fl = raw.loc[raw["wwtp_jurisdiction"] == "Florida"].copy()
sum = fl[fl["date_start"] == "2022-04-16"].sum()["population_served"]
percent = (sum / counties[counties["fips"].str[:2] == "12"].sum()["population"]) * 100
print(f"Coverage for {sum} in FL, or {round(percent,2)}% of FL's total population")


In [ ]:
fpl_data = fpl_data.merge(counties, on="fips", how="left")
# fpl_data.sort_values("date_start", ascending=False)
# fpl_data.dropna().sort_values("date_start", ascending=False)

In [ ]:

# FPL Locations
fpl_sumamry = fpl_data[fpl_data["date_start"] == "2022-04-16"].groupby(["fips", "county_names"]).sum()[["population_served", "population"]]
fpl_sumamry["percent_county_coverage"] = round(fpl_sumamry['population_served'] / fpl_sumamry['population'] * 100, 1)
fpl_sumamry["percent_county_coverage"] = fpl_sumamry["percent_county_coverage"].astype(str) + "%"
fpl_sumamry.rename(columns={
    "population_served": "Population coverage",
    "population": "Total population",
    "percent_county_coverage": "Percent of total county population covered"
})

In [ ]:
site_metadata = raw[raw["date_start"] == "2022-04-16"][["key_plot_id", "county_fips", "county_names", "population_served"]].drop_duplicates().dropna()
# site_metadata

# Normalized Data

In [ ]:
nwss = pd.read_csv("https://data.cdc.gov/api/views/g653-rqe2/rows.csv?accessType=DOWNLOAD")
normalized_nwss = nwss.merge(site_metadata, on="key_plot_id").rename(columns={"county_fips": "fips"})
normalized_nwss = normalized_nwss.merge(counties, on="fips", how="left")

In [ ]:
nd = normalized_nwss.dropna()
nd.loc[nd["fips"].isin(fpl_counties)].copy().sort_values("date", ascending=False)